## Modern `dict` syntax

### `dict` Comprehensions (python3)

In [1]:
dial_codes = [
    (880, 'Bangladesh'),
    (55, 'Brazil'),
    (86, 'China'),
    (91, 'India'),
    (62, 'Indonesia'),
    (81, 'Japan'),
    (234, 'Nigeria'),
    (92, 'Pakistan'),
    (7, 'Russia'),
    (1, 'United States')
]

country_dial = {country: code for code, country in dial_codes}
country_dial

{'Bangladesh': 880,
 'Brazil': 55,
 'China': 86,
 'India': 91,
 'Indonesia': 62,
 'Japan': 81,
 'Nigeria': 234,
 'Pakistan': 92,
 'Russia': 7,
 'United States': 1}

In [2]:
{code: country.upper()
    for country, code in sorted(country_dial.items())
    if code < 70}

{55: 'BRAZIL', 62: 'INDONESIA', 7: 'RUSSIA', 1: 'UNITED STATES'}

### Unpacking Mappings (python 3.5)

In [3]:
def dump(**kwargs):
    return kwargs
dump(**{'x': 1}, y=2, **{'z': 3})

{'x': 1, 'y': 2, 'z': 3}

In [4]:
{'a': 0, **{'x': 1}, 'y': 2, **{'z': 3, 'x': 4}}

{'a': 0, 'x': 4, 'y': 2, 'z': 3}

### Merging Mappings with | (python 3.9)

In [5]:
d1 = {'a': 1, 'b': 3}
d2 = {'a': 2, 'b': 4, 'c': 6}
d1 | d2

{'a': 2, 'b': 4, 'c': 6}

In [6]:
d1

{'a': 1, 'b': 3}

In [7]:
d1 |= d2
d1

{'a': 2, 'b': 4, 'c': 6}

## Pattern Matching with Mappings
The `match/case` statement supports subjects that are mapping objects. Thanks to destructuring, *pattern matching is a powerful tool to process records structured like nested mappings and sequences, which we often need to read from JSON APIs* and databases with semi-structured schemas, like MongoDB, EdgeDB, or PostgreSQL.


In [8]:
def get_creators(record: dict) -> list:
    match record:
        case {'type': 'book', 'api': 2, 'authors': [*names]}:
            return names
        case {'type': 'book', 'api': 1, 'author': name}:
            return [name]
        case {'type': 'book'}:
            raise ValueError(f"Invalid 'book' record: {record!r}")
        case {'type': 'movie', 'director': name}:
            return [name]
        case _:
            raise ValueError(f'Invalid record: {record!r}')


In [9]:
b1 = dict(api=1, author='Douglas Hofstadter',
        type='book', title='Gödel, Escher, Bach')
get_creators(b1)

['Douglas Hofstadter']

In [12]:
from collections import OrderedDict
b2 = OrderedDict(api=2, type='book', title='Python in a Nutshell',
            authors=['Martelli', 'Ravenscroft', 'Holden'])
get_creators(b2)

['Martelli', 'Ravenscroft', 'Holden']

In [13]:
get_creators({'type': 'book', 'pages': 770})

ValueError: Invalid 'book' record: {'type': 'book', 'pages': 770}

In [14]:
food = dict(category='ice cream', flavor='vanilla', cost=199)
match food:
    case {'category': 'ice cream', **details}:
        print(f'Ice cream details: {details}')

Ice cream details: {'flavor': 'vanilla', 'cost': 199}


## Standard API of Mapping Types
The `collections.abc` module provides the `Mapping` and `MutableMapping` ABCs describing the interfaces of `dict` and similar types
<img src="../images/7.png" style="width: 70%;">

To implement a custom mapping, it’s easier to extend `collections.UserDict`, or to wrap a dict by composition, instead of subclassing these ABCs. The `collections.UserDict` class and all concrete mapping classes in the standard library encapsulate the basic dict in their implementation, which in turn is built on a hash table. Therefore, they all share the limitation that *the keys must be hashable (the values need not be hashable, only the keys)*

In [16]:
tt = (1, 2, (30, 40))
print(f"{hash(tt) = }")

hash(tt) = -3907003130834322577


In [18]:
tl = (1, 2, [30, 40])
hash(tl)

TypeError: unhashable type: 'list'

User-defined types are hashable by default because their hash code is their `id()`, and the `__eq__()` method inherited from the object class simply compares the object IDs.  
If an object implements a custom `__eq__()` that takes into account its internal state, it will be hashable only if its `__hash__()` always returns the same hash code. In practice, this requires that `__eq__()` and `__hash__()` only take into account instance attributes that never change during the life of the object.

### Overview of Common Mapping Methods

<img src="../images/8.png" style="width: 80%;">

`my_dict.setdefault(key, []).append(new_value)` is equal to
```python
if key not in my_dict:
    my_dict[key] = []
my_dict[key].append(new_value)
```
except that the latter code performs at least two searches for key — or even three if it’s not found — while `setdefault` does it all with a single lookup.

In [24]:
my_dict = {}
my_dict.setdefault("a", []).append(1)
print(my_dict)
my_dict.setdefault("a", []).append(2)
print(my_dict)

{'a': [1]}
{'a': [1, 2]}


## Automatic Handling of Missing Keys
Sometimes it is convenient to have mappings that return some made-up value when a missing key is searched. There are two main approaches to this: 
1. Use a `defaultdict` instead of a plain `dict`. 2
2. Subclass `dict` or any other mapping type and add a `__missing__` method

### `defaultdict`: Another Take on Missing Keys
A `collections.defaultdict` instance creates items with a default value on demand whenever a missing key is searched using `d[k]` syntax

In [36]:
import collections
from typing import DefaultDict

class A:
    def __init__(self) -> None:
        print("init A")

d: DefaultDict[str, A] = collections.defaultdict(A)
d["a1"]
print(d)

init A
defaultdict(<class '__main__.A'>, {'a1': <__main__.A object at 0x7f43212cba60>})


### The `__missing__` Method
Underlying the way mappings deal with missing keys is the aptly named `__missing__` method. This method is not defined in the base dict class, but dict is aware of it: if you subclass dict and provide a `__missing__` method, the standard `dict.__getitem__` will call it whenever a key is not found, instead of raising KeyError.

In [54]:
class StrKeyDict0(dict):
    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]
    
    def get(self, key, default=None):
        try:
            return self[key]  # calls __getitem__ which calls __missing__
        except KeyError:
            return default
    
    def __contains__(self, key):
        return key in self.keys() or str(key) in self.keys()
    
d = StrKeyDict0()
d['2'] = 'two'
d[2]

'two'

In [58]:
2 in d

True

## Variations of `dict`

### `collections.OrderedDict`

`dict` vs `OrderedDict`:
- The equality operation for `OrderedDict` checks for matching order.  
- The `popitem()` method of `OrderedDict` has a different signature. It accepts an optional argument to specify which item is popped.
- `OrderedDict` has a `move_to_end()` method to efficiently reposition an element to an endpoint.
- **The regular `dict` was designed to be very good at mapping operations**. Tracking insertion order was secondary.
- **`OrderedDict` was designed to be good at reordering operations**. Space efficiency, iteration speed, and the performance of update operations were secondary.
- Algorithmically, `OrderedDict` can handle frequent reordering operations better than `dict`. This makes it suitable for **tracking recent accesses** (for example, in an LRU cache).

### `collections.ChainMap`

A `ChainMap` instance holds a list of mappings that can be searched as one. The lookup is performed on each input mapping in the order it appears in the constructor call, and succeeds as soon as the key is found in one of those mappings

In [3]:
d1 = dict(a=1, b=3)
d2 = dict(a=2, b=4, c=6)
from collections import ChainMap
chain = ChainMap(d1, d2)
print(f"{chain['a'] = }")
print(f"{chain['c'] = }")

chain['a'] = 1
chain['c'] = 6


The `ChainMap` instance does not copy the input mappings, but holds references to them. Updates or insertions to a `ChainMap` only affect the first input mapping

In [4]:
chain['c'] = -1
d1

{'a': 1, 'b': 3, 'c': -1}

### `collections.Counter`

`collections.Counter` is a mapping that holds an integer count for each key. Updating an existing key adds to its count. This can be used to count instances of hashable objects or as a multiset

In [6]:
import collections

ct = collections.Counter('abracadabra')
ct

Counter({'a': 5, 'b': 2, 'r': 2, 'c': 1, 'd': 1})

In [7]:
ct.update('aaaaazzz')
ct

Counter({'a': 10, 'z': 3, 'b': 2, 'r': 2, 'c': 1, 'd': 1})

In [8]:
ct.most_common(3)

[('a', 10), ('z', 3), ('b', 2)]

### `shelve.Shelf`
The `shelve` module in the standard library provides persistent storage for a mapping of string keys to Python objects serialized in the pickle binary forma

### Subclassing `UserDict` Instead of `dict`
It’s better to subclass `UserDict` rather than `dict` since the built-in has some implementation shortcuts that end up forcing us to override methods that we can just inherit from `UserDict` with no problems. Note that `UserDict` does not inherit from `dict`, but uses composition: it has an internal `dict` instance, called data, which holds the actual items to avoid undesired recursion when coding special methods like `__setitem__`, and simplifies the coding of `__contains__`

Example: `StrKeyDict` always converts nonstring keys to str on insertion, update, and lookup

In [9]:
import collections

class StrKeyDict(collections.UserDict):
    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]
    
    def __contains__(self, key):
        return str(key) in self.data
    
    def __setitem__(self, key, item):
        self.data[str(key)] = item


## Immutable Mappings

The mapping types provided by the standard library are all mutable, but you may need to prevent users from changing a mapping by accident. The types module provides a wrapper class called `MappingProxyType`, which, given a mapping, returns a `mappingproxy` instance that is a read-only but dynamic proxy for the original mapping. This means that updates to the original mapping can be seen in the `mappingproxy`, but changes cannot be made through it

In [10]:
from types import MappingProxyType
d = {1: 'A'}
d_proxy = MappingProxyType(d)
d_proxy

mappingproxy({1: 'A'})

In [11]:
d_proxy[2] = 'x'

TypeError: 'mappingproxy' object does not support item assignment

In [12]:
d[2] = 'B'

In [13]:
d_proxy

mappingproxy({1: 'A', 2: 'B'})

## Dictionary Views

Dictionary views allow high-performance operations on a `dict`, without unnecessary copying of data. The `dict` instance methods `.keys(), .values(), and .items()` return instances of classes called `dict_keys`, `dict_values`, and `dict_items`, respectively